In [1]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import matplotlib as mpl
import math as m
from tqdm.notebook import tqdm
import illustris_python as il
import sys
from scipy.stats import binned_statistic_2d
import matplotlib.colors as colors

import six
import glob
import os
from illustris_python.groupcat import gcPath, offsetPath
from illustris_python.util import partTypeNum
from illustris_python.sublink import treePath, treeOffsets, subLinkOffsets, loadTree, maxPastMass

SnapPath="/n/holylfs05/LABS/hernquist_lab/IllustrisTNG/Runs/L35n2160TNG/output"
h=0.6774 

In [27]:
def numMergers(tree, minMassRatio=1e-10, massPartType='stars', index=0):
    """ Calculate the number of mergers, along the main progenitor branch, in this sub-tree 
    (optionally above some mass ratio threshold).  """
    # verify the input sub-tree has the required fields
    reqFields = ['SubhaloID', 'NextProgenitorID', 'MainLeafProgenitorID',
                 'FirstProgenitorID', 'SubhaloMassType']

    if not set(reqFields).issubset(tree.keys()):
        raise Exception('Error: Input tree needs to have loaded fields: '+', '.join(reqFields))

    num = 0
    invMassRatio = 1.0 / minMassRatio

    # walk back main progenitor branch
    rootID = tree['SubhaloID'][index]
    fpID   = tree['FirstProgenitorID'][index]


    fpIndex = index + (fpID - rootID)
    fpMass  = maxPastMass(tree, fpIndex, massPartType)
    print('fp snapnum=', tree['SnapNum'][fpIndex])

    # explore breadth
    npID = tree['NextProgenitorID'][fpIndex]


    while npID != -1:
        npIndex = index + (npID - rootID)
        npMass  = maxPastMass(tree, npIndex, massPartType)

        print('snapnum ', tree['SnapNum'][npIndex])
        
        # count if both masses are non-zero, and ratio exceeds threshold
        if fpMass > 0.0 and npMass > 0.0:
            ratio = npMass / fpMass

            if ratio >= minMassRatio and ratio <= invMassRatio:
                print('npID', npID, 'fpID', fpID)
                num += 1


        npID = tree['NextProgenitorID'][npIndex]


    return num

For the galaxy we are looking at, its subhalo ID is 159068 at Snapshot=30 (and 137385 at Snapshot=29). Let's use this as an example

In [28]:
treeMPB = il.sublink.loadTree(SnapPath, 30, 159068, onlyMPB=True)

In [29]:
print(treeMPB['SnapNum'][:5])
print(treeMPB['SubfindID'][:5])

[30 29 28 27 26]
[159068 137385 129193 116174 104507]


In [30]:
tree = il.sublink.loadTree(SnapPath,30,159068)
print(tree['SnapNum'][0], tree['SubfindID'][0], tree['SubhaloID'][0])
numMergers(tree, minMassRatio=0.001)

30 159068 18843240
fp snapnum= 29
snapnum  28
npID 18861994 fpID 18843241
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  28
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  28
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29
snapnum  29


1

In [18]:
key = tree['SnapNum']==29
print(tree['SubfindID'][key])
print(tree['SubhaloID'][key])

[ 137385  137540 3535500  137499 2294799  137478  137518  137467  137508
  137521 1990677 3268724  137538  137516  937059  137485  137535  137490
  937062 5551158 4259212 3255577  137532 3717931 3889247  137513 2308925
  137530 3660382 6152733 5140544 6730305 4229407 4988204 6840052 4600175
 5831662]
[18843241 18862537 18862595 18862630 18862670 18862699 18862725 18862751
 18862779 18862794 18862815 18862840 18862858 18862883 18862902 18862917
 18862935 18862958 18862969 18862982 18863005 18863012 18863019 18863028
 18863035 18863041 18863043 18863046 18863051 18863053 18863055 18863057
 18863059 18863061 18863062 18863063 18863064]


In [32]:
tree = il.sublink.loadTree(SnapPath,30,159068)

id_match = 18861994
k = np.where(tree['SubhaloID'] == id_match)
print(tree['SnapNum'][k], tree['SubfindID'][k])

id_match = 18843241
k = np.where(tree['SubhaloID'] == id_match)
print(tree['SnapNum'][k], tree['SubfindID'][k])

[28] [129197]
[29] [137385]


In [27]:
SnapNum = 30
id_match = 18861994
tree = il.sublink.loadTree(SnapPath, SnapNum, 
                           159068,
                           fields=['NextProgenitorID','SubfindID','SnapNum'])
for i in range(0, len(tree['NextProgenitorID'])):
    subfind_id = tree['SubfindID'][i]
    next_prog_id = tree['NextProgenitorID'][i]
    sn = tree['SnapNum'][i]
    if next_prog_id == id_match:
        print('subfind_id', subfind_id, sn)

subfind_id 137385 29


In [31]:
SnapNum = 30
id_match = 18843241
tree = il.sublink.loadTree(SnapPath, SnapNum, 
                           159068,
                           fields=['FirstProgenitorID','SubfindID','SnapNum'])
for i in range(0, len(tree['FirstProgenitorID'])):
    subfind_id = tree['SubfindID'][i]
    next_prog_id = tree['FirstProgenitorID'][i]
    sn = tree['SnapNum'][i]
    if next_prog_id == id_match:
        print('subfind_id', subfind_id, sn)

subfind_id 159068 30


In [18]:
SnapNum = 29
id_match = 18861994
tree = il.sublink.loadTree(SnapPath, SnapNum, 
                           137385,
                           fields=['NextProgenitorID','SubfindID', 'SnapNum'])
for i in range(0, len(tree['NextProgenitorID'])):
    subfind_id = tree['SubfindID'][i]
    next_prog_id = tree['NextProgenitorID'][i]
    sn = tree['SnapNum'][i]
    if next_prog_id == id_match:
        print('subfind_id', subfind_id, sn)

subfind_id 137385 29


In [12]:
il.groupcat.loadSingle(SnapPath, 29, subhaloID=137385)['SubhaloMass']/h

51.894546927763784

In [13]:
il.groupcat.loadSingle(SnapPath, 30, subhaloID=137385)['SubhaloMass']/h

0.008167432258711263

In [30]:
numMergers(il.sublink.loadTree(SnapPath,22,51227), minMassRatio=0.001)

npID 18849543 fpID 18843249
npID 18849594 fpID 18843249


2

In [20]:
SnapNum = 22
id_match = 18849543
tree = il.sublink.loadTree(SnapPath, SnapNum, 
                           51227,
                           fields=['NextProgenitorID','SubfindID','SnapNum'])
for i in range(0, len(tree['NextProgenitorID'])):
    subfind_id = tree['SubfindID'][i]
    next_prog_id = tree['NextProgenitorID'][i]
    sn = tree['SnapNum'][i]
    if next_prog_id == id_match:
        print('subfind_id', subfind_id, sn)

subfind_id 41552 21
